In [1]:
from dotenv import load_dotenv 
load_dotenv()

True

In [2]:
from langchain.document_loaders import DirectoryLoader 
from langchain.document_loaders import PyPDFLoader 
data_folder = "../data"  

loader = DirectoryLoader(data_folder, glob="*.pdf", loader_cls=PyPDFLoader)
documents = loader.load()


In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter 
splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=100)

In [4]:
chunks=splitter.split_documents(documents)

In [5]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


C:\Users\saipr\AppData\Local\Temp\ipykernel_28688\3920331017.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\saipr\BioRag\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from langchain.vectorstores import FAISS 
vectorstore=FAISS.from_documents(chunks,embeddings)


In [7]:
from langchain_groq import ChatGroq 
import os 
GROQ_API_KEY=os.getenv('GROQ_API_KEY')

llm=ChatGroq(api_key=GROQ_API_KEY,model='llama-3.3-70b-versatile')

In [8]:
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever

retriever_dense = vectorstore.as_retriever(search_kwargs={"k": 10})  
bm25_retriever = BM25Retriever.from_documents(chunks)

hybrid_retriever = EnsembleRetriever(retrievers=[retriever_dense, bm25_retriever], weights=[0.5, 0.5])


In [9]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful and knowledgeable biology tutor. Answer clearly and accurately.if the query is out of sylabus just give Out of syllabus"),
    ("human", "Context:\n{context}\n\nQuestion: {question}")
])


In [10]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    memory_key="chat_history",
    return_messages=True,
    k=3
)


C:\Users\saipr\AppData\Local\Temp\ipykernel_28688\1796011903.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(


In [11]:
from typing import List
from langchain.schema import Document, BaseRetriever 
from sentence_transformers import CrossEncoder
from pydantic import BaseModel  
from langchain.chains import ConversationalRetrievalChain
reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

def rerank_documents(query: str, retrieved_docs: List[Document]) -> List[Document]:
    docs_texts = [doc.page_content for doc in retrieved_docs]
    pairs = [(query, doc_text) for doc_text in docs_texts]
    scores = reranker.predict(pairs)
    sorted_docs = [doc for _, doc in sorted(zip(scores, retrieved_docs), key=lambda x: x[0], reverse=True)]
    return sorted_docs

class RerankRetriever(BaseRetriever, BaseModel): 
    base_retriever: BaseRetriever  
    top_k: int = 5  

    def _get_relevant_documents(self, query: str) -> List[Document]:
        initial_docs = self.base_retriever.get_relevant_documents(query)
        reranked_docs = rerank_documents(query, initial_docs)
        return reranked_docs[:self.top_k]  

base_retriever = EnsembleRetriever(retrievers=[retriever_dense, bm25_retriever], weights=[0.5, 0.5])


custom_retriever = RerankRetriever(base_retriever=base_retriever, top_k=5)

qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=custom_retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": prompt}
)
query="Write two chemical required to conduct an experiment to prove that carbon dioxide is necessary for photosynthesis"
result = qa_chain({"question": query})
print(result["answer"])

C:\Users\saipr\AppData\Local\Temp\ipykernel_28688\253123294.py:36: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"question": query})
C:\Users\saipr\AppData\Local\Temp\ipykernel_28688\253123294.py:20: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  initial_docs = self.base_retriever.get_relevant_documents(query)


To prove that carbon dioxide is necessary for photosynthesis, two chemicals required are:

1. Potassium hydroxide (KOH) - to absorb carbon dioxide
2. Iodine - to test the presence of starch in the leaves.


In [12]:
result['chat_history']

[]